In [49]:
import pandas as pd
import numpy as np
import re
import nltk
import string 
import preprocessor as p 
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
import torch 

In [55]:
input_data = []

with open('./sts-2017-en-es/En_Es_STS/STS.input.en-es.train.txt', 'r') as inFile:
    input_data = inFile.readlines()

X = []
for data in input_data:
    X.append(data.split('\t')[:2])

In [ ]:
class Data(to)

In [72]:
def sentence_preprocessor(data):
    cleaned_data = []
    for x in data:
        cleaned_x = p.clean(x).lower()
        for c in string.punctuation:
            if c in cleaned_x:
                cleaned_x = cleaned_x.replace(c,'')
        tokenized_x = word_tokenize(cleaned_x)
        cleaned_data.append(tokenized_x)
    
    return cleaned_data

def vocab_maker(data):
    vocab = {}
    index=0
    for sentence in data:
        for word in sentence:
            if word in vocab:
                continue
            else:
                vocab[word] = index
                index+=1
    
    return vocab

def vectorizer(data, vocab):
    x = np.zeros((len(data),len(vocab.keys())))
    for index in range(len(data)):
        sentence = data[index]
        for word in sentence:
            x[index][vocab[word]] += 1
    return x

In [64]:
eng_data, esp_data = tuple(zip(*X))

cleaned_eng = sentence_preprocessor(eng_data)
cleaned_esp = sentence_preprocessor(esp_data)


In [67]:
# make vocab
eng_vocab = vocab_maker(cleaned_eng)
esp_vocab = vocab_maker(cleaned_esp)


In [76]:
# generating embeddings
eng_vectors = vectorizer(cleaned_eng, eng_vocab)
esp_vectors = vectorizer(cleaned_esp, esp_vocab)

In [80]:
eng_vectors.shape
esp_vectors.shape

(1000, 4863)